In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


# Train 

- predict the best horse in a given race


### TODO
- try to predict a given distribution according to ranks, instead of the first one
- try to predict best among all combinaisons

In [2]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors, probability_at_rank, return_against
import winning_validation
from winning_horse_models import sklearn
from winning_horse_models.logistic_regression import LogisticRegressionModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel

SOURCE = "PMU"

In [3]:
sequential_sgd_regression = LogisticRegressionModel.load_model()

In [4]:
returns = return_against.compute_return_against_odds(source="PMU",same_races_support=True, winning_model=sequential_sgd_regression, verbose=False);

In [5]:
return_against.compute_overall_average(returns)

(0.04917593446047697,
 0.4947343446536749,
 -0.2528736465547953,
 -2.098604165707568)

In [6]:
returns

{'source': 'PMU',
 'same_races_support': True,
 'winning_model': 'LogisticRegressionModel',
 'min_horse': 2,
 'max_horse': 26,
 'n_horses_return_against_odds': {2: {'predicted_return_against_odds': array([-0.92798346]),
   'random_predicted_return_against_odds': array([-0.83394534]),
   'race_ps_notna_index': array([ True]),
   'race_ids': [81655],
   'message': 'Comparing on all races w/ 2 horses (1 races in total)'},
  3: {'predicted_return_against_odds': array([ 0.4221329 , -0.51021155,  0.14850537,  0.77141609, -0.70852701,
           3.01234561,  0.01545975]),
   'random_predicted_return_against_odds': array([ 1.48696981,  1.3985934 , -0.81761999,  1.09460597, -0.50968196,
           5.55164319, -1.41380378]),
   'race_ps_notna_index': array([False,  True,  True, False,  True,  True,  True,  True,  True]),
   'race_ids': [81032,
    81112,
    83398,
    84456,
    86809,
    89774,
    94479,
    94644,
    102856],
   'message': 'Comparing on same races w/ 3 horses with odds 7 r

In [9]:
exact_top_1 = errors.compute_validation_error(source=SOURCE, 
                                                          k=1,
                                                          winning_model=sequential_sgd_regression, 
                                                          validation_method=errors.exact_top_k)
winning_validation.compute_overall_average(exact_top_1)

(0.23681652490886998, 0.09218307006885379, 0.27112191170514377)

In [6]:
kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=sequential_sgd_regression, 
                                                            validation_method=winning_validation.kappa_cohen_like)

In [7]:
precision_at_3 = errors.compute_validation_error(source=SOURCE, 
                                                             k=3,
                                                             winning_model=sequential_sgd_regression, 
                                                             validation_method=winning_validation.precision_at_k)

(0.23681652490886998, 0.09218307006885379, 0.27112191170514377)

In [9]:
winning_validation.compute_overall_average(kappa_cohen_1)

(0.15875357464821155, -0.0014250643409744832, 0.19765694181213814)

In [10]:
winning_validation.compute_overall_average(precision_at_3)

(0.43272982569835416, 0.2813263666680168, 0.45352181133298225)

In [8]:
rank_race, predicted_probabilities, base_probabilities=(np.arange(1,5).reshape((1,4)), np.array([1,0,0,0]).reshape((1,4)), np.ones((1,4,))*1/4)

In [9]:
ssert rank_race.shape == predicted_probabilities.shape == base_probabilities.shape

SyntaxError: invalid syntax (<ipython-input-9-0bb82a9bc34b>, line 1)

In [17]:
n_horses =3
source="PMU"
winning_model=sequential_sgd_regression
from winning_horse_models.baselines import RandomModel
same_races_support=True
from winning_validation.return_against import *

In [12]:
x_race, rank_race, race_dfs = import_data.get_races_per_horse_number(
            source=source,
            n_horses=n_horses,
            on_split="val",
            x_format="sequential_per_horse",
            y_format="rank",
        )

In [18]:
previous_stakes = np.stack(
            arrays=[race_df["totalEnjeu"].values for race_df in race_dfs], axis=0
        )

model_prediction = winning_model.predict(x=x_race)
random_prediction = RandomModel().predict(x=x_race)
race_ps_notna_index = np.logical_not(np.isnan(previous_stakes)).all(axis=1)
rectified_pari_mutual_probas = get_rectified_pari_mutual_probabilities(
    previous_stakes=np.nan_to_num(previous_stakes)  # Put nan to zero
)
rank_race_ = rank_race

if same_races_support:
    model_prediction = model_prediction[race_ps_notna_index]
    random_prediction = random_prediction[race_ps_notna_index]
    rectified_pari_mutual_probas = rectified_pari_mutual_probas[
        race_ps_notna_index
    ]
    rank_race_ = rank_race_[race_ps_notna_index]


return_against = np.where(
rank_race_ == 1,
model_prediction / rectified_pari_mutual_probas,
np.zeros_like(rank_race_),
)
    
return_against /= np.sum(rank_race_ == 1, axis=1,keepdims=True)

In [19]:
from constants import PMU_MINIMUM_BET_SIZE

previous_stakes=PMU_MINIMUM_BET_SIZE*np.ones((1,4))
pari_mutual_odds = expected_return.get_race_odds(
        track_take=0.0,
        previous_stakes=previous_stakes,
        race_bet=PMU_MINIMUM_BET_SIZE * np.ones_like(previous_stakes),
    )  # To avoid infinite return
print(1 / pari_mutual_odds)

[[0.00166667 0.00166667 0.00166667 0.00166667]]


In [15]:
np.sum(rank_race_ == 1, axis=1).reshape(rank_race_.shape)

NameError: name 'rank_race_' is not defined

In [21]:
previous_stakes.shape

(9, 3)

In [24]:
get_rectified_pari_mutual_probabilities(
        previous_stakes=np.nan_to_num(previous_stakes)  # Put nan to zero
    ).shape

(9, 3)

In [29]:
race_ps_notna_index

array([False,  True,  True, False,  True,  True,  True,  True,  True])

In [30]:
rank_race_.shape

(7, 3)

In [28]:
model_prediction.shape

(7, 3)

In [8]:
xgboost_winning_model = XGBoostWinningModel.load_model(trainable=False)

In [9]:
exact_top_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=xgboost_winning_model, validation_method=winning_validation.exact_top_k)

/home/mathieu/.pyenv/versions/3.7.4/envs/venv_benter/lib/python3.7/site-packages/joblib-0.15.1-py3.7.egg/joblib/memory.py:531: DtypeWarning: Columns (35,46,47,61,62,63,64,87) have mixed types.Specify dtype option on import or set low_memory=False.
  out, metadata = self.call(*args, **kwargs)


For races w/ 3 horses, 9 races in val, top 1 in right order: 55.556% (Random: 22.222%, Odds 55.556%)
For races w/ 4 horses, 77 races in val, top 1 in right order: 31.169% (Random: 35.065%, Odds 50.649%)
For races w/ 5 horses, 354 races in val, top 1 in right order: 33.616% (Random: 16.949%, Odds 44.350%)
For races w/ 6 horses, 829 races in val, top 1 in right order: 23.643% (Random: 16.888%, Odds 36.188%)
For races w/ 7 horses, 1489 races in val, top 1 in right order: 24.312% (Random: 14.909%, Odds 37.139%)
For races w/ 8 horses, 1914 races in val, top 1 in right order: 24.922% (Random: 11.755%, Odds 35.005%)
For races w/ 9 horses, 2072 races in val, top 1 in right order: 22.683% (Random: 12.403%, Odds 33.687%)
For races w/ 10 horses, 2457 races in val, top 1 in right order: 18.030% (Random: 10.623%, Odds 30.322%)
For races w/ 11 horses, 2269 races in val, top 1 in right order: 18.070% (Random: 9.079%, Odds 28.735%)
For races w/ 12 horses, 2740 races in val, top 1 in right order: 17.95

In [3]:
lgmb_winning_model = LGBMWinningModel.load_model()

In [4]:
exact_top_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=lgmb_winning_model, validation_method=winning_validation.exact_top_k)

For races w/ 3 horses, 9 races in val, top 1 in right order: 44.444% (Random: 22.222%, Odds 55.556%)
For races w/ 4 horses, 77 races in val, top 1 in right order: 29.870% (Random: 35.065%, Odds 50.649%)
For races w/ 5 horses, 354 races in val, top 1 in right order: 31.356% (Random: 16.949%, Odds 44.350%)
For races w/ 6 horses, 829 races in val, top 1 in right order: 25.814% (Random: 16.888%, Odds 36.188%)
For races w/ 7 horses, 1489 races in val, top 1 in right order: 26.998% (Random: 14.909%, Odds 37.139%)
For races w/ 8 horses, 1914 races in val, top 1 in right order: 27.691% (Random: 11.755%, Odds 35.005%)
For races w/ 9 horses, 2072 races in val, top 1 in right order: 22.442% (Random: 12.403%, Odds 33.687%)
For races w/ 10 horses, 2457 races in val, top 1 in right order: 20.228% (Random: 10.623%, Odds 30.322%)
For races w/ 11 horses, 2269 races in val, top 1 in right order: 19.392% (Random: 9.079%, Odds 28.735%)
For races w/ 12 horses, 2740 races in val, top 1 in right order: 20.07

In [3]:

for SklearnModel in [
     sklearn.DecisionTreeModel, sklearn.SVCModel, sklearn.KNNModel,
                     sklearn.RandomForestModel,
                     sklearn.GradientBoostingModel, sklearn.GaussianNBModel, sklearn.LDAModel,
                     sklearn.SGDModel
                    ]:
    print(SklearnModel.__name__)
    sklearn_winning_model = SklearnModel.load_model()
    kappa_cohen_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=sklearn_winning_model, 
                                                              validation_method=winning_validation.kappa_cohen_like)
    print(winning_validation.compute_overall_average(kappa_cohen_1))
    print()

DecisionTreeModel
(0.030470628793524007, 0.002130053016071174, 0.19769219098749824)
SVCModel
(0.04685053088385765, 0.002130053016071174, 0.19769219098749824)
KNNModel
(0.1837746083693874, 0.002130053016071174, 0.19769219098749824)
RandomForestModel
(0.09921375068151626, 0.002130053016071174, 0.19769219098749824)
GradientBoostingModel
(0.11398277168714965, 0.002130053016071174, 0.19769219098749824)
GaussianNBModel
(-0.0008610628552462091, 0.002130053016071174, 0.19769219098749824)
LDAModel
(0.06514285743523787, 0.002130053016071174, 0.19769219098749824)
SGDModel
(0.05170616861614152, 0.002130053016071174, 0.19769219098749824)


/home/mathieu/.pyenv/versions/3.7.4/envs/venv_benter/lib/python3.7/site-packages/scikit_learn-0.23.1-py3.7-linux-x86_64.egg/sklearn/linear_model/_base.py:327: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
